In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import os
import matplotlib.pyplot as plt

import sklearn
from sklearn.svm import SVC

from sklearn.metrics import balanced_accuracy_score, roc_auc_score, accuracy_score
from scipy.stats import zscore, pearsonr,uniform
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, StratifiedKFold,RandomizedSearchCV

from scipy.io import loadmat
from scipy.stats import ttest_ind

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
train_cat=pd.read_excel("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TRAIN/TRAIN_CATEGORICAL_METADATA.xlsx")
train_cat.head()

In [ ]:
train_cat.columns

In [ ]:
train_FCM=pd.read_csv("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TRAIN//TRAIN_FUNCTIONAL_CONNECTOME_MATRICES.csv")
train_FCM.head()

In [ ]:
train_FCM.columns

In [ ]:
train_Quant=pd.read_excel("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TRAIN/TRAIN_QUANTITATIVE_METADATA.xlsx")
train_Quant.head()

In [ ]:
train_Quant.columns

In [ ]:
train_Solutions=pd.read_excel("C:/Users/PC2019-1/OneDrive/Desktop/widsdatathon2025/TRAIN/TRAINING_SOLUTIONS.xlsx")
train_Solutions.head()

In [ ]:
train_Solutions.columns

In [ ]:
train_cat.info()

In [ ]:
train_cat.describe()

In [ ]:
train_cat['Barratt_Barratt_P2_Occ'].value_counts()

In [ ]:
sns.countplot(x='Barratt_Barratt_P2_Occ', data=train_cat[['Barratt_Barratt_P2_Occ']])
plt.title(f"Distribution of Barratt_Barratt_P2_Occ")
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.countplot(x='Barratt_Barratt_P1_Occ', data=train_cat[['Barratt_Barratt_P1_Occ']])
plt.title(f"Distribution of Barratt_Barratt_P1_Occ")
plt.xticks(rotation=45)
plt.show()

In [ ]:
data = train_cat[['Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Occ']]

# Pretvaranje u dug oblik
melted_data = pd.melt(data, var_name='Category', value_name='Answers')

# Grafikon
plt.figure(figsize=(8, 4))
sns.countplot(x='Answers', hue='Category', data=melted_data)
plt.title("Distribution of P1_Occ and P2_Occ")
plt.xticks(rotation=45)
plt.legend(title='Columns')
plt.show()

In [ ]:
stacked_data = data.apply(pd.Series.value_counts).fillna(0)

# Grafikon
stacked_data.plot(kind='bar', stacked=True, figsize=(8, 4), colormap='viridis')
plt.title("Stacked Distribution of P1_Occ and P2_Occ")
plt.xlabel("Answers")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.legend(title='Columns')
plt.show()

In [ ]:
g = sns.FacetGrid(melted_data, col='Category', sharey=True, height=5, aspect=1.5)
g.map(sns.countplot, 'Answers', order=data.stack().value_counts().index)
g.set_titles("{col_name}")
g.set_xticklabels(rotation=45)
plt.show()

In [ ]:
data = train_cat[['Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P2_Edu']]

# Pretvaranje u dug oblik
melted_data = pd.melt(data, var_name='Category', value_name='Answers')

# Grafikon
plt.figure(figsize=(8, 4))
sns.countplot(x='Answers', hue='Category', data=melted_data)
plt.title("Distribution of P1_Edu and P2_Edu")
plt.xticks(rotation=45)
plt.legend(title='Columns')
plt.show()

In [ ]:
train_Quant['MRI_Track_Age_at_Scan'].hist(figsize=(10,6), bins=20)
plt.suptitle("MRI_Track_Age_at_Scan Distributions")
plt.xlabel('MRI_Track_Age_at_Scan')
plt.ylabel('Frequency Count')
plt.show()

In [ ]:
train_Solutions['ADHD_Outcome'].value_counts()

In [ ]:
ax=train_Solutions['ADHD_Outcome'].value_counts().plot(kind='bar', color='blue')
plt.title('ADHD_Outcome')
plt.xlabel('Outcome (1 = YES, 0 = NO )')
plt.ylabel('Count')
plt.xticks(rotation=0)
# Dodavanje vrednosti iznad svakog bara
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                textcoords='offset points')
plt.show()

In [ ]:
train_Solutions['Sex_F'].value_counts()

In [ ]:
ax=train_Solutions['Sex_F'].value_counts().plot(kind='bar', color=['red', 'blue'])
plt.title('Sex_F')
plt.xlabel('Outcome (1 = Female, 0 = Male )')
plt.ylabel('Count')
plt.xticks(rotation=0)
# Dodavanje vrednosti iznad svakog bara
for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', fontsize=10, color='black', xytext=(0, 5),
                textcoords='offset points')
plt.show()

In [ ]:
train_Quant.columns

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(train_Quant['SDQ_SDQ_Emotional_Problems'], kde=True, color='skyblue')
plt.title('Distribution of SDQ_SDQ_Emotional_Problems')
plt.xlabel('SDQ_SDQ_Emotional_Problems')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Using Cohens coefficient we can measure correlational strength between 2 columns
# We will be using this method to see how much of an impact each column of Quantitaive Metadata has on Trainting Solutions
def validate_correlation_strength(columns_to_test, column_to_examine = ""):
    for column in columns:
        # ----------------- PART 1 -----------------
        # Box plot
        plt.figure(figsize=(8, 6))
        sns.boxplot(x=column_to_examine, y= column, data=train_Quant_copy)
        plt.title(f'{column} vs {column_to_examine}')
        plt.ylabel(column_to_examine)
        plt.ylabel(column)
        plt.show()
    
        # ----------------- PART 2 -----------------
        # Splitting data into 2 groups
        group_0 = train_Quant_copy[train_Quant_copy[column_to_examine] == 0][column] # group with no ADHD
        group_1 = train_Quant_copy[train_Quant_copy[column_to_examine] == 1][column] # group with ADHD
        
        # T-test - Students test
        t_stat, p_value = ttest_ind(group_0, group_1)
        print(f"T-statistic: {t_stat}, P-value: {p_value}")
        
        # Testira hipotezu:
        # 𝐻0: Srednje vrednosti su iste između ADHD i non-ADHD grupa.
        # 𝐻1: Postoji razlika u srednjim vrednostima.
        
        # Interpretacija
        if p_value < 0.05:
            print("Postoji značajna razlika u srednjim vrednostima između grupa.")
        else:
            print("Nema značajne razlike u srednjim vrednostima između grupa.")
        print("-"*60)
    
        # ----------------- PART 3 -----------------    
        # Ako otkrijemo značajne razlike između grupa, možemo izračunati efekat veličine kako bismo kvantifikovali koliko su razlike velike:
        # Cohen's d: Mera efekta za T-test.
        
        # d < 0.2 – Vrlo mala razlika (zanemarljiv efekat)
        # 0.2 ≤ d < 0.5 – Mala razlika (slab efekat)
        # 0.5 ≤ d < 0.8 – Srednja razlika (umeren efekat)
        # d ≥ 0.8 – Velika razlika (jak efekat)
        
        mean_diff = np.mean(group_1) - np.mean(group_0)
        pooled_std = np.sqrt((np.std(group_1)**2 + np.std(group_0)**2) / 2)
        cohen_d = mean_diff / pooled_std
        print(f"Cohen's d: {cohen_d}")
        print("_"*120)
    
        cohens_value_dict[column] = cohen_d
    return cohens_value_dict

In [ ]:
# TESTING CORRELATION STRENGTHS FOR:
    # DOCUMENT: TRAIN_SOLUTIONS
    # COLUMN NAME: 'ADHD Outcome'

# Making a shallow copy.
train_Quant_copy=train_Quant.copy()
# Assigning a new column 'ADHD_Outcome' to the train_Quant_copy from train_Solutions
train_Quant_copy['ADHD_Outcome']=train_Solutions['ADHD_Outcome']

# Assigning all the columns to the list of columns names
# Excluding the participant_id column
columns = train_Quant.columns[1:]

# We will store Cohens coefficient values here for every column of train_Quant_copy
cohens_value_dict = {}
cohens_value_dict = validate_correlation_strength(columns, 'ADHD_Outcome')
# Printing cohens values
cohens_value_dict

In [ ]:
# TESTING CORRELATION STRENGTHS FOR:
    # DOCUMENT: TRAIN_SOLUTIONS
    # COLUMN NAME: 'ADHD Outcome'

# Making a shallow copy.
train_Quant_copy=train_Quant.copy()
# Assigning a new column 'ADHD_Outcome' to the train_Quant_copy from train_Solutions
train_Quant_copy['Sex_F']=train_Solutions['Sex_F']

# Assigning all the columns to the list of columns names
# Excluding the participant_id column
columns = train_Quant.columns[1:]

# We will store Cohens coefficient values here for every column of train_Quant_copy
cohens_value_dict = {}
cohens_value_dict = validate_correlation_strength(columns, 'Sex_F')
# Printing cohens values
cohens_value_dict

In [ ]:
# ANALYSING CATEGORICAL DATA WITH ADHD_OUTCOME AND SEX_F

train_cat_copy = train_cat.copy()
train_cat_copy['ADHD_Outcome'] = train_Solutions['ADHD_Outcome']

# Assigning all the columns to the list of columns names
for column in columns:
    sns.countplot(data = train_cat, x = column, hue = train_Solutions['ADHD_Outcome'])
    plt.title(f'ADHD Prevalence by {column}')
    plt.show()
    print(train_cat[column].value_counts())
    adhd_percentages = train_cat_copy.groupby(column)['ADHD_Outcome'].mean()
    print('-'*60)
    print(adhd_percentages)
    print('_'*120)

In [ ]:
# ANALYSING CATEGORICAL DATA WITH ADHD_OUTCOME AND SEX_F

train_cat_copy = train_cat.copy()
train_cat_copy['Sex_F'] = train_Solutions['Sex_F']

# Assigning all the columns to the list of columns names
for column in columns:
    sns.countplot(data = train_cat, x = column, hue = train_Solutions['Sex_F'])
    plt.title(f'Sex_F Prevalence by {column}')
    plt.show()
    print(train_cat[column].value_counts())
    adhd_percentages = train_cat_copy.groupby(column)['Sex_F'].mean()
    print('-'*60)
    print(adhd_percentages)
    print('_'*120)

In [ ]:
for col in train_cat.select_dtypes(include='int').columns:
    train_cat[col]=train_cat[col].astype('category')

In [ ]:
    # Creating a list of all of the columns except the first participant_id
    columns_to_encode=train_cat.columns[1:].tolist()
    # Print the columns to encode
    print("Columns to encode;", columns_to_encode)

In [ ]:
data_encoded=pd.get_dummies(train_cat[columns_to_encode],drop_first=True)
data_encoded=data_encoded.map(lambda x:1 if x is True else (0 if x is False else x))

In [ ]:
#Combine encoded columns with the rest of the DataFrame
cat_train_final=pd.concat([train_cat.drop(columns=columns_to_encode), data_encoded], axis=1)
cat_train_final.head()